# VAR Model

## Problem Statement

Predict electricity prices in Spain for each hour of the upcoming day more accurately than estimates provided by the Spanish transmission agent and operator. 

Use information available during the 2pm-3pm window the previous day during which generators in Spain submit their bids. 

## Contents

### Imports

In [1]:
# General Imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

# General modeling imports
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, TimeSeriesSplit
from sklearn import metrics

# Models
from statsmodels.tsa.api import VAR

In [2]:
df = pd.read_csv('../Data/Analysis/model_data.csv')
visuals = pd.read_csv('../Data/intermediary/energy.csv')

In [3]:
# visuals contains the prices attached to their original hour
# and will be used to visualize and compare our predictions
visuals.set_index(pd.DatetimeIndex(visuals['time']), inplace=True)
visuals = visuals[['price_actual','price_day_ahead']]
visuals.head(2)

,price_actual,price_day_ahead
time,,
2015-01-01 00:00:00+00:00,65.41,50.1
2015-01-01 01:00:00+00:00,64.92,48.1


### Functions Used

In [4]:
# Function for evaluating the regressions
# and outputting a dataframe of different metrics
# for each hour predicted
def reg_metrics(y_train, y_train_p, y_test, y_test_p, mod):
    test_rmse = np.sqrt(((y_test-y_test_p)**(2)).mean())
    train_r2 = metrics.r2_score(y_train, y_train_p, multioutput='raw_values')
    test_r2 =  metrics.r2_score(y_test, y_test_p, multioutput='raw_values')
    metrics_df = pd.DataFrame(data = zip(test_rmse, train_r2, test_r2),
                              columns=[mod+'test_rmse',mod+'train_r2',mod+'test_r2'])
    return metrics_df

In [5]:
# function to convert predictions into dataframe for plotting
def append_preds(preds, previous_preds, name):
    new_preds = pd.DataFrame(np.ravel(preds),columns=[name], index=previous_preds.index)
    return previous_preds.join(new_preds)

### Prepare Data

In [6]:
# Set up data frame for modeling
# Drop time column
df.drop(columns=['time'], inplace=True)
# set index as date
df.set_index(pd.DatetimeIndex(df['date']), inplace=True)
# sort index
df.sort_index(inplace=True)
# drop hour of day and date column
df.drop(columns=['hour_of_day','date'], inplace=True)

# Get columns for y
y_cols = [col for col in df.columns if col.startswith('t_price')]

# only need y for VAR model
y = df[y_cols]

# Train test split
train, test = train_test_split(y, shuffle=False)

### VAR Model

In [16]:
# Instantiate a VAR model
model = VAR(train)

# Fit our model and use AIC
var_model = model.fit(maxlags=14, # at most a lag of 2 weeks
                     ic='aic')  # using AIC to determine the best model

# What is the order of our autoregressive model? 
var_model.k_ar

/Users/magnusbigelow/Applications/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:162: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


7

In [20]:
# Generate predictions
var_model.forecast(test, steps=2)

KeyError: -1